In [1]:
#Import Libraries

import os
from itertools import chain
import json
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.io.json import json_normalize
import seaborn as sns
import nltk

In [51]:
path = r"/content/tweets_ws22_v1.json"

with open(path) as f:
   
      dump = json.loads(f.read())
      org_df = pd.json_normalize(dump)
      org_df
    
final_df = pd.DataFrame() 

for i in range(0,len(dump)):
    temp_topic_df = pd.json_normalize(dump[i]['labels']['topic'])
    temp_sentiment_df = pd.json_normalize(dump[i]['labels']['sentiment'])
    temp_topic_df.rename(columns={'id':'topic_id','tweet_id':'topic_tweet_id','user_id':'topic_user_id','ground_truth':'topic_ground_truth'}, inplace = True)
    temp_sentiment_df.rename(columns={'id':'sentiment_id','tweet_id':'sentiment_tweet_id','user_id':'sentiment_user_id','ground_truth':'sentiment_ground_truth'}, inplace = True)
    df_join_topic_sent = temp_topic_df.join(temp_sentiment_df)
    final_df = final_df.append(df_join_topic_sent)
    #final_df = pd.concat([org_df,Joined_df],axis = 1)
    
    
cols = final_df.columns
org_df[cols] = final_df[cols].values


data =  org_df.drop(columns=['source_created_at','longitude','latitude','labels.topic','labels.sentiment','topic_tweet_id','sentiment_tweet_id','author_id','source','topic_user_id','topic_ground_truth','language','id','source_id','relevant','sentiment_ground_truth','sentiment_id','topic_id','sentiment_user_id'])
data

,text,topic,sentiment
0,@DSisourath The Thameslink core between London...,service,negative
1,@DulwichHistory Loving the complaint about peo...,delays,negative
2,@SW_Help .And yet you have no toilets on some ...,toilets,negative
3,@SW_Help you have no toilets on some of your t...,toilets,negative
4,@SpeedySticks007 @MrNeilJH @TLRailUK @christia...,seats,neutral
...,...,...,...
16944,Haha oh man the audio corruption on @TLRailUK ...,announcements,neutral
16945,@TLRailUK SweetIs there a plug to charge my ph...,plugs,neutral
16946,@TLRailUK now there are far fewer commuters ha...,tables,neutral
16947,@geofftech I am voting for Thameslink. 1. in a...,toilets,neutral


In [54]:
df = data[(data.sentiment == "negative") & (data.topic == "delays")]

In [55]:
df['text'].iloc[1]

"@TLRailUK @SW_Help @SouthernRailUK @TfL Yyyiouuu FUCKING CUNTS.Just once it'd be nice to know my cunting train isn't cancelled for some cunting reason. You're the reason Britain is a third world country when it comes to trains. Piss poor service, expensive as fuck. GET FUCKKKKED!"

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5851 entries, 1 to 16843
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5851 non-null   object
 1   topic      5851 non-null   object
 2   sentiment  5851 non-null   object
dtypes: object(3)
memory usage: 182.8+ KB


In [57]:
#pip install bertopic

In [58]:
from bertopic import BERTopic

In [59]:
topic_model =BERTopic(embedding_model = "all-MiniLM-L6-v2")

In [60]:
Text = df['text'].tolist()

In [61]:
Text[0]

'@DulwichHistory Loving the complaint about people having to wait 10 minutes for a train.They clearly never travelled via Thameslink.'

In [62]:

topics, probs = topic_model.fit_transform(Text)

In [63]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1660,-1_the_tlrailuk_for_and
1,0,647,0_trains_tlrailuk_the_train
2,1,447,1_thameslink_the_for_of
3,2,258,2_plat_northern_great_train
4,3,135,3_repay_delay_claim_you
...,...,...,...
78,77,12,77_haywardsheath_burgesshill_sussexincidents_s...
79,78,12,78_reopened_lines_albans_st
80,79,12,79_claim_was_15_my
81,80,11,80_surrey_sutton_albans_city


In [64]:
topic_model.get_topic(0)

[('trains', 0.01479501277697147),
 ('tlrailuk', 0.014757740973197812),
 ('the', 0.01450472062977359),
 ('train', 0.012624301502329687),
 ('you', 0.011405884880183358),
 ('it', 0.011083304911978571),
 ('for', 0.010247197179126108),
 ('that', 0.009907920221476177),
 ('this', 0.009729295124350305),
 ('in', 0.009247356670610237)]

In [65]:
import pandas as pd
df = pd.DataFrame({"topic2": topics, "text2": Text})

In [66]:
df

,topic2,text2
0,1,@DulwichHistory Loving the complaint about peo...
1,0,@TLRailUK @SW_Help @SouthernRailUK @TfL Yyyiou...
2,0,@TLRailUK announced a cancelled train > 2 mins...
3,0,"""Congestion"" is NOT a reason for delay, it's a..."
4,-1,"""Govia Thameslink Railway (GTR) is facing a £5..."
...,...,...
5846,-1,Thameslink at Bedford; en route to Elstree for...
5847,62,The 7.15 wrh to lbg train is ALWAYS delayed by...
5848,10,Well done @TLRailUK again 😠 8:16 SAC to Bright...
5849,1,What a total joke on starting my hen do @LNER ...


In [73]:
topic_model.visualize_topics(top_n_topics= 50)

In [68]:
topic_model.visualize_barchart()

In [69]:
topic_model.visualize_heatmap()

In [70]:
topic_model.visualize_term_rank()

In [71]:
topic_model.visualize_hierarchy()